In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')

In [3]:
train_df = pd.DataFrame(train)
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
train_df = train_df.drop(['PassengerId', 'Ticket', 'Cabin'], axis = 1)

In [5]:
train_df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [6]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.NaN, strategy = 'mean')
train_df["Age"] = imputer.fit_transform(train_df[["Age"]])

In [7]:
train_df.dropna(inplace = True)

In [8]:
train_df.isnull().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [9]:
train_df = train_df.drop(['Name'], axis = 1)

In [10]:
train_df["Family"] = train_df["SibSp"] + train_df["Parch"]
train_df = train_df.drop(['SibSp'], axis = 1)

In [11]:
train_df["New_Result"] = train_df["Fare"] / train_df["Age"]

In [12]:
X = train_df.iloc[:, 1:]
y = train_df.iloc[:, 0]

In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X["Sex"] = le.fit_transform(X["Sex"])

In [14]:
one_hot =pd.get_dummies(X["Embarked"])
X = pd.concat([X, one_hot], axis = 1)

In [15]:
X = X.drop(['Embarked'], axis = 1)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state = 0)

In [18]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [81]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report


classifier = MLPClassifier(hidden_layer_sizes=(50, 50), max_iter=700, random_state=0, 
                      solver = 'adam', activation = 'tanh',alpha = 0.01,
                     learning_rate = 'adaptive')

# Fitting the model
classifier.fit(X_train, y_train)       #0.45

MLPClassifier(activation='tanh', alpha=0.01, hidden_layer_sizes=(50, 50),
              learning_rate='adaptive', max_iter=700, random_state=0)

In [82]:
y_pred = classifier.predict(X_test)

In [83]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7415730337078652

In [84]:
test = pd.read_csv('/kaggle/input/titanic/test.csv')
test_df = pd.DataFrame(test)

In [85]:
id = test_df["PassengerId"]
test_df = test_df.drop(['PassengerId', 'Ticket', 'Cabin'], axis = 1)

In [86]:
test_df.isnull().sum()

test_df["Age"] = imputer.fit_transform(test_df[["Age"]])
test_df["Fare"] = imputer.fit_transform(test_df[["Fare"]])                                        

In [87]:
test_df = test_df.drop(['Name'], axis = 1)
test_df["Family"] = test_df["SibSp"] + test_df["Parch"]
test_df = test_df.drop(['SibSp'], axis = 1)  
test_df["New_Result"] = test_df["Fare"]/ test_df["Age"]

In [88]:
test_df["Sex"] = le.fit_transform(test_df["Sex"])

one_hot_2 =pd.get_dummies(test_df["Embarked"])
test_df = pd.concat([test_df, one_hot_2], axis = 1)

test_df = test_df.drop(['Embarked'], axis = 1)

In [89]:
test_df = sc.transform(test_df)

In [90]:
test_predictions = classifier.predict(test_df)

In [92]:
submission_df = pd.DataFrame({'PassengerId' : id, 'Survived' : test_predictions})
submission_df.to_csv('titanic_pred.csv', index = False)